In [1]:
import bs4
import requests
import os
import re
import os
import datetime
import json
import codecs

In [2]:
# download main page to get download links
page = requests.get('https://catalog.unc.edu/courses/')
f = open('/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/courses.html', "wb")
f.write(page.content)
f.close()
headers = page.headers
print(headers)

{'Date': 'Wed, 11 May 2022 02:42:22 GMT', 'Server': 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips', 'Last-Modified': 'Tue, 25 Jan 2022 21:53:40 GMT', 'ETag': '"b143-5d66f1e3f1865"', 'Accept-Ranges': 'bytes', 'Content-Length': '45379', 'X-Frame-Options': 'SAMEORIGIN', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=UTF-8'}


In [3]:
# get links to all course pages
file = open('/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/courses.html', 'r')
html = bs4.BeautifulSoup(file, 'html5lib')
file.close()

categories = []
links = html.find('ul', {'id': '/courses/'}).find_all('a')
links = list(links)
for link in links:
  categories.append(link['href'])

categoryLinks = []
for category in categories:
  categoryLinks.append(f'https://catalog.unc.edu{category}')

In [4]:
# download the category pages
if len(categoryLinks) == len(categories):
  for i in range(len(categories)):
    page = requests.get(categoryLinks[i])
    f = open(f'/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/{categories[i][:-1]}.html', "wb")
    f.write(page.content)
    f.close()

In [ ]:
# scrape course catalog
directory = '/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/courses'
output = {}
for file in os.listdir(directory):
  f = os.path.join(directory, file)
  if os.path.isfile(f):
    print(f'\n{datetime.datetime.now()} current file: {file}')
    page = open(f, 'r')
    html = bs4.BeautifulSoup(page, 'html5lib')
    page.close()
    courses = list(html.findAll('div', {'class': 'courseblock'}))
    for course in courses:
      try:
        rawTitle = course.findAll('p', {'class': 'courseblocktitle'})
        rawTitle = rawTitle[0].stripped_strings
        for title in rawTitle:
          # get raw title text and clean it
          title = repr(title)
          if 'xa0' in title:
            title = title.replace('\\xa0', ' ')
          title = re.sub(r'(\\n)', ' ', title)
          title = re.sub('\s+', ' ', title)
          title = title.split('.')

          # get course code
          courseCode = title[0].strip().split('\'')[1].replace(' ', '')

          # get course title
          courseTitle = title[1].strip()

          # get number of credit hours awarded
          creditHoursAwarded = title[2].strip()
        
        # get raw text of description and clean it
        rawDesc = course.findAll('p', {'class': 'courseblockdesc'})
        rawDesc = rawDesc[0].get_text()
        rawDesc = re.sub(r'(\\n)', ' ', rawDesc)
        rawDesc = re.sub('\s+', ' ', rawDesc)

        # get course description
        try:
          if 'Gen Ed:' in rawDesc:
            courseDesc = re.split(r'Gen Ed:', rawDesc)
            courseDesc = courseDesc[0].strip()
        except:
          print(f'no course description found for {courseCode}')
          courseDesc = 'none'

        # get making connections requirement info
        try:
          if 'Repeat rules:' in rawDesc:
            makingConnections = re.split(r'Gen Ed:(.+?\.)', rawDesc)
            makingConnections = makingConnections[1].strip()
        except:
          print(f'no making connections information found for {courseCode}')
          makingConnections = 'none'

        # get repeat rules:
        try:
          if 'Repeat rules:' in rawDesc:
            repeatRules = re.split(r'Repeat rules:(.+?)Grading', rawDesc)
            repeatRules = repeatRules[1].strip()
        except:
          print(f'no repeat rules found for {courseCode}')
          repeatRules = 'none'
      
        # get grading status rules 
        try:
          if 'Grading status:' in rawDesc:
            gradingStatus = re.split(r'Grading status:(.+?\.)', rawDesc)
            gradingStatus = gradingStatus[1].strip()
        except:
          print(f'no repeat rules found for {courseCode}')
          gradingStatus = 'none'

        data = {
          'category': file,
          'course code': courseCode,
          'course title': courseTitle,
          'credit hours awarded': creditHoursAwarded,
          'description': courseDesc,
          'making connections requirement': makingConnections,
          'ideas in action requirement': '',
          'repeat rules': repeatRules,
          'grading status': gradingStatus,
        }
        # print(courseCode)
        # print(data)
        output[courseCode] = data
      except:
        print(f'an unknown error occured with {file} at {datetime.datetime.now()}')
print(output)

In [14]:
# get the ideas in action information into the data
directory = '/Users/aniruddhdoki/Library/Mobile Documents/com~apple~CloudDocs/school/NCSSM/senior year/semester 2/cs4280 advanced java/cs4280-final-project/data/course pages/ideas in action'
output = {}
for file in os.listdir(directory):
  f = os.path.join(directory, file)
  if os.path.isfile(f):
    print(f'\n{datetime.datetime.now()} current file: {file}')
    page = open(f, 'r')
    html = bs4.BeautifulSoup(page, 'html5lib')
    page.close()

    table = html.find('tbody', {'class': 'wpv-loop js-wpv-loop'})
    print(table.get_text())
    


2022-05-10 22:49:19.943094 current file: 23.html

		
			
				EXSS 420
Program Planning in Recreation Services
HI-SERVICE

			
		
			
				EXSS 493
Field Experience in Sport Administration
HI-INTERN

			
		
			
				EXSS 50
First-Year Seminar: Discrimination and Sport
FY-SEMINAR

			
		
			
				EXSS 51
First-Year Seminar: Entrepreneurship in Human Performance and Sport
FY-SEMINAR

			
		
			
				EXSS 593
Practicum in Physical Fitness and Wellness
HI-INTERN

			
		
			
				EXSS 693H
Senior Honors Thesis
HNRS, RESEARCH

			
		
			
				EXSS 694H
Senior Honors Thesis
HNRS, RESEARCH

			
		
			
				EXSS 89
First-Year Seminar: Special Topics
FY-SEMINAR

			
		
			
				FOLK 202
Introduction to Folklore
FC-AESTH, FC-KNOWING
ANTH 202, ENGL 202
			
		
			
				FOLK 310
Fairy Tales
FC-AESTH
ENGL 310
			
		
			
				FOLK 370
Southern Legacies: The Descendants Project
FC-POWER, RESEARCH
ANTH 370
			
		
			
				FOLK 375
Southern Food Studies
FC-KNOWING, FC-POWER
AMST 375
			
		
			
				FOLK 380
Traditio

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte